# Day 4 Statistical Functions and Random Data
## Part 1: 기술 통계 함수
### 1.1 기본 통계량의 수학적 이해 

In [9]:
import numpy as np
rng = np.random.default_rng(42) # RNG : Random Number Generator, (seed)
data = rng.normal(loc = 100, scale = 15, size = 1000) # 정규분포에서 샘플을 뽑아, mean은 100, std은 15, 개수는 1000인 것.

In [5]:
print(data.mean())
print(data.std())

99.5666267350608
14.830835330947618


In [6]:
mean = np.mean(data)
median = np.median(data)
# 최빈값 사용 원하면 mode = stats.mode(data)

print(f"Mean: {mean:.4f}")
print(f"Median: {median:.4f}")

# 평균 - 중앙값 : skewness hint
print(f"Mean - Median = {mean - median:.4f}")


Mean: 99.5666
Median: 100.0927
Mean - Median = -0.5260


### 1.2 DDOF :  분산 / 표준편차

In [ ]:
# ===== ===== ===== ===== ===== ===== ===== =====
# 산포도 (Dispersion)
# ===== ===== ===== ===== ===== ===== ===== =====

data = np.array([2, 4, 6, 8, 10])

# ddof : Delta Degrees of Freedom
# 모분산 : Population Variance : N으로 나눔
var_pop = np.var(data, ddof = 0)

# sample variance (불편추정량) : N-1로 나눔
# 표본분산은 평균을 추정하느라 자유도 1을 잃었기 때문에, N−1로 나눠야 모분산을 과소추정하지 않는다!
var_sample = np.var(data, ddof = 1)

print(f"모분산 (ddof=0): {var_pop:.4f}")
print(f"표본분산 (ddof=1): {var_sample:.4f}")


# 표준편차 = sqrt(분산)
std_pop = np.std(data, ddof=0)
std_sample = np.std(data, ddof=1)
print(f"모표준편차: {std_pop:.4f}")
print(f"표본표준편차: {std_sample:.4f}")


모분산 (ddof=0): 8.0000
표본분산 (ddof=1): 10.0000
모표준편차: 2.8284
표본표준편차: 3.1623


Q/A : 왜 ddof = 1을 써야 할까?

In [11]:
# test
np.random.seed(42)
population = np.random.normal(100, 15, size = 100000)
true_variance = np.var(population, ddof = 0)
print(f"모집단의 분산 : {true_variance:.2f}")

모집단의 분산 : 225.41


In [16]:
# 표본 크기 30으로 1000번 추출해보자.
n_experiments = 1000
sample_size = 30
var_ddof0 = []
var_ddof1 = []

rng = np.random.default_rng(42)
for _ in range(n_experiments):
    sample = rng.choice(population, size=sample_size, replace=False)
    var_ddof0.append(np.var(sample, ddof=0))
    var_ddof1.append(np.var(sample, ddof=1))
    
print(f"\nddof=0 평균 추정치: {np.mean(var_ddof0):.2f} (편향됨, 과소추정)")
print(f"ddof=1 평균 추정치: {np.mean(var_ddof1):.2f} (정확)")


ddof=0 평균 추정치: 217.68 (편향됨, 과소추정)
ddof=1 평균 추정치: 225.19 (정확)


### 1.3 분위수 / 백분위수
- Quantiles / Percentiles

1. 지수분포의 수학적 정의
- 지수분포(exponential distribution)의 확률밀도함수는 다음과 같다.

$$
f(x;\lambda) = \lambda e^{-\lambda x}, \quad x \ge 0
$$
그리고

$\lambda$ : rate parameter (단위 시간당 발생률)

---
2. 평균과 분산

- 지수분포의 이론적 성질은 다음과 같다.

$$
\mathbb{E}[X] = \frac{1}{\lambda}
$$

$$
\mathrm{Var}(X) = \frac{1}{\lambda^2}
$$

---
3. NumPy의 파라미터화 방식

NumPy는 $$\lambda$$ 대신 **scale**을 사용한다.

$$
\text{scale} = \frac{1}{\lambda}
$$

따라서

```python
rng.exponential(scale=10)
```

은 다음과 동등하다.

$$
\lambda = 0.1
$$

---

4. `scale`의 통계적 의미

지수분포에서는 다음 관계가 성립한다.

$$
\text{mean} = \text{std} = \text{scale}
$$

즉,

* 평균 = scale
* 표준편차 = scale

In [17]:
rng = np.random.default_rng(42)
data = rng.exponential(scale = 10, size = 1000)

In [18]:
p25 = np.percentile(data, 25)
p50 = np.percentile(data, 50)
p75 = np.percentile(data, 75)
p95 = np.percentile(data, 95)

print(f"25th percentila (Q1): {p25:.2f}")
print(f"50th percentila (Q2): {p50:.2f}")
print(f"75th percentila (Q3): {p75:.2f}")
print(f"95th percentila (Q4): {p95:.2f}")

25th percentila (Q1): 2.91
50th percentila (Q2): 6.80
75th percentila (Q3): 14.10
95th percentila (Q4): 30.88


In [19]:
# quantile의 범위를 0~1로 맞춘다면?
q1, q2, q3 = np.quantile(data, [0.25, 0.5, 0.75])
print(f"\nQuantiles: Q1={q1:.2f}, Q2={q2:.2f}, Q3={q3:.2f}")


Quantiles: Q1=2.91, Q2=6.80, Q3=14.10


In [20]:
# IQR (Interquartile Range) : 이상치 탐지
iqr = q3 - q1
lower_bound = q1-1.5*iqr
upper_bound = q3 + 1.5*iqr

outliers = data[(data < lower_bound) | (data > upper_bound)]

print(f"\nIQR: {iqr:.2f}")
print(f"이상치 범위: < {lower_bound:.2f} 또는 > {upper_bound:.2f}")
print(f"이상치 개수: {len(outliers)} ({100*len(outliers)/len(data):.1f}%)")


IQR: 11.19
이상치 범위: < -13.87 또는 > 30.88
이상치 개수: 50 (5.0%)


### 1.4 여러 분위 수 한 번에 계산해보기

In [23]:
percentiles = [4, 11, 23, 40, 60, 77, 89, 96]
values = np.percentile(data, percentiles)
print(values)

[ 0.4489727   1.08995916  2.57941637  4.97179247  9.52111088 14.8463589
 22.72879329 33.07953539]


In [25]:
for p, v in zip(percentiles, values):
    print(f"{p}th percentils : {v:8.2f}")

4th percentils :     0.45
11th percentils :     1.09
23th percentils :     2.58
40th percentils :     4.97
60th percentils :     9.52
77th percentils :    14.85
89th percentils :    22.73
96th percentils :    33.08


## Part 2 : axis parameter

### 2.1 Axis 직관적으로 이해하기
- 이 축을 따라 값들을 합친다!

In [27]:
arr = np.array([[1, 2, 3],
                [4, 5, 6]])
print(arr)
print(f"shape: {arr.shape}")

[[1 2 3]
 [4 5 6]]
shape: (2, 3)


In [28]:
# axis = 0 : 행으로 합쳐봅시다.
col_sum = np.sum(arr, axis=0)
print(f"axis = 0 : {col_sum}")
print(f"shape : {col_sum.shape}")

axis = 0 : [5 7 9]
shape : (3,)


In [30]:
# axis = 1 : 열로도 합쳐봅시다.
row_sum = np.sum(arr, axis = 1)
print(row_sum)
print(row_sum.shape)

[ 6 15]
(2,)


In [31]:
# 기본값
total = np.sum(arr)
total

np.int64(21)

### 2.2 3차원 배열

In [35]:
# rng = np.random.default_rng(42)
# rng.uniform : continuous uniform distribution에서 난수 생성. /
# X ~ u(15, 30)에서 모든 값이 동일 확률 발생한다는 뜻.
sst = rng.uniform(15, 30, size=(12, 5, 6)) # (12개월, 위도, 경도)
time_mean = np.mean(sst, axis = 0)
print(time_mean.shape)
global_mean = np.mean(sst)
print(f"{global_mean:.2f}")

(5, 6)
22.82


### 2.3 keepdims 로 차원 유지하기

In [36]:
data = np.array([[10, 20, 30],
                 [40, 50, 60],
                 [70, 80, 90]])

# 행별 평균 (with keepdims = False)
rowmean1 = np.mean(data, axis = 1)
print(rowmean1)
print(rowmean1.shape)

[20. 50. 80.]
(3,)


In [38]:
rowmean2 = np.mean(data, axis = 1, keepdims=True)
print(rowmean2)
print(f"{rowmean2.shape}")

[[20.]
 [50.]
 [80.]]
(3, 1)


In [39]:
deviations = data - rowmean2
print(deviations)

[[-10.   0.  10.]
 [-10.   0.  10.]
 [-10.   0.  10.]]


### 2.4 Z-score (x - mean) / std

In [41]:
# rng = np.random.default_rng(42)
# 온도 15-30 / 염분 33-37 / 클로로필 0.1-10

raw_data = np.column_stack([
    rng.uniform(15, 30, 100),
    rng.uniform(33, 37, 100),
    rng.uniform(0.1, 10, 100)
])
print(raw_data[:5])

[[17.1154095  33.89578266  5.54820271]
 [29.58519372 34.14756965  0.87374578]
 [28.54339031 34.00173235  8.51291306]
 [28.8284626  34.05148533  6.17689973]
 [19.98244572 35.32977897  6.18589377]]


In [42]:
raw_data.shape

(100, 3)

In [43]:
# 각 변수(col)별로 평균 / 표준편차 계산?
col_mean = np.mean(raw_data, axis=0, keepdims=True)
col_std = np.std(raw_data, axis=0, keepdims=True, ddof = 1)

print(f"평균 : {col_mean}")
print(f"표준편차 : {col_std}")

평균 : [[22.55355955 34.99314163  4.86832476]]
표준편차 : [[4.2322445  1.21675856 2.83516781]]


In [44]:
print(f"\n변수별 평균: {col_mean.flatten()}")
print(f"변수별 표준편차: {col_std.flatten()}")


변수별 평균: [22.55355955 34.99314163  4.86832476]
변수별 표준편차: [4.2322445  1.21675856 2.83516781]


In [45]:
z_score = (raw_data - col_mean) / col_std 
print(z_score[:5])

print(f"표준화 후 평균 : {np.mean(z_score, axis=0)}")
print(f"표준화 후 표준편차: {np.std(z_score, axis=0, ddof=1)}")

[[-1.28493287 -0.90187075  0.23980166]
 [ 1.66144328 -0.69493817 -1.40893917]
 [ 1.41528467 -0.8147954   1.28549297]
 [ 1.48264191 -0.77390563  0.46155115]
 [-0.60750598  0.27666733  0.46472346]]
표준화 후 평균 : [1.62092562e-16 1.06381570e-14 1.89631297e-16]
표준화 후 표준편차: [1. 1. 1.]


## Part 3: 상관관계와 공분산
### 3.1 Correlation Coefficient

In [46]:
# ===== ===== ===== ===== ===== ===== ===== =====
# 피어슨 상관계수 : 두 변수 간 선형 관계 강도
# ===== ===== ===== ===== ===== ===== ===== =====

# rng = np.random.default_rng(42)
n = 100

# create data
x = rng.normal(0, 1, n)
noise = rng.normal (0, 0.1, n)

y_pos = 2 * x + noise
y_neg = -3 * x + noise
y_none = rng.normal(0, 1 ,n)

corr_matrix = np.corrcoef(x, y_pos)
print(corr_matrix)

[[1.         0.99892457]
 [0.99892457 1.        ]]


In [47]:
all_vars = np.vstack([x, y_pos, y_neg, y_none])
corr_all = np.corrcoef(all_vars)
print(np.around(corr_all, 3))

[[ 1.     0.999 -1.     0.018]
 [ 0.999  1.    -0.997  0.02 ]
 [-1.    -0.997  1.    -0.017]
 [ 0.018  0.02  -0.017  1.   ]]


### 3.2 Covariance
- Cov(X, Y) = E[(X-ux)(Y-uy)]

In [52]:
cov_matrix = np.cov(x, y_pos)
print(f" 공분산 행렬 :\n {cov_matrix}")

 공분산 행렬 :
 [[0.94010557 1.88706998]
 [1.88706998 3.79606812]]
